Let's start here! If you can directly link to an image relevant to your notebook, such as [canonical logos](https://github.com/numpy/numpy/blob/main/doc/source/_static/numpylogo.svg), do so here at the top of your notebook. You can do this with Markdown syntax,

> `![<image title>](http://link.com/to/image.png "image alt text")`

or edit this cell to see raw HTML `img` demonstration. This is preferred if you need to shrink your embedded image. **Either way be sure to include `alt` text for any embedded images to make your content more accessible.**

<img src="images/ProjectPythia_Logo_Final-01-Blue.svg" width=250 alt="Project Pythia Logo"></img>

# Crash Course Towards Your First Model

Next, title your notebook appropriately with a top-level Markdown header, `#`. Do not use this level header anywhere else in the notebook. Our book build process will use this title in the navbar, table of contents, etc. Keep it short, keep it descriptive. Follow this with a `---` cell to visually distinguish the transition to the prerequisites section.

---

## Overview
If you have an introductory paragraph, lead with it here! Keep it short and tied to your material, then be sure to continue into the required list of topics below,

1. This is a numbered list of the specific topics
1. These should map approximately to your main sections of content
1. Or each second-level, `##`, header in your notebook
1. Keep the size and scope of your notebook in check
1. And be sure to let the reader know up front the important concepts they'll be leaving with

## Prerequisites
This section was inspired by [this template](https://github.com/alan-turing-institute/the-turing-way/blob/master/book/templates/chapter-template/chapter-landing-page.md) of the wonderful [The Turing Way](https://the-turing-way.netlify.app) Jupyter Book.

Following your overview, tell your reader what concepts, packages, or other background information they'll **need** before learning your material. Tie this explicitly with links to other pages here in Foundations or to relevant external resources. Remove this body text, then populate the Markdown table, denoted in this cell with `|` vertical brackets, below, and fill out the information following. In this table, lay out prerequisite concepts by explicitly linking to other Foundations material or external resources, or describe generally helpful concepts.

Label the importance of each concept explicitly as **helpful/necessary**.

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |
| Project management | Helpful | |

- **Time to learn**: estimate in minutes. For a rough idea, use 5 mins per subsection, 10 if longer; add these up for a total. Safer to round up and overestimate.
- **System requirements**:
    - Populate with any system, version, or non-Python software requirements if necessary
    - Otherwise use the concepts table above and the Imports section below to describe required packages as necessary
    - If no extra requirements, remove the **System requirements** point altogether

---

## Imports

Here we'll be using a basic set of Python libraries, along with Numba for fast numerical routines. A helpful constants file is also imported.

In [1]:
import numpy as np
import numba
import xarray as xr
import matplotlib.pyplot as plt

from constants import *

## Introduction

...TODO...

In [ ]:
# some subsection code
new = "helpful information"

## Basic Equations

...TODO...

### Starting Equations

...TODO...

### Assumptions and Simplification

...TODO...

### Final Prognostic Equations

...TODO...

## Model Configuration

...TODO...

### Grid Type

...TODO...

### Initial/Boundary Conditions

...TODO...

## Discritization

...TODO...

In [2]:
@numba.njit()
def u_tendency_u_advection_term(u, dx):
    term = np.zeros_like(u)
    for k in range(1, u.shape[0] - 1):
        for i in range(1, u.shape[1] - 1):
            term[k, i] = u[k, i] * (u[k, i + 1] - u[k, i - 1]) / (2 * dx)
    return term

In [4]:
@numba.njit()
def u_tendency_w_advection_term(u, w, dz):
    term = np.zeros_like(u)
    for k in range(1, u.shape[0] - 1):
        for i in range(1, u.shape[1] - 1):
            term[k, i] = 0.25 * (
                w[k + 1, i] + w[k + 1, i - 1] + w[k, i] + w[k, i - 1] 
            ) * (u[k + 1, i] - u[k - 1, i]) / (2 * dz)
    return term

In [5]:
@numba.njit()
def u_tendency_pgf_term(u, pi, theta_base, dx):
    term = np.zeros_like(u)
    for k in range(1, u.shape[0] - 1):
        for i in range(1, u.shape[1] - 1):
            term[k, i] = c_p * theta_base[k] * (pi[k, i] - pi[k, i - 1]) / dx
    return term

In [35]:
@numba.njit()
def u_tendency(u, w, pi, theta_base, dx, dz):
    return (
        u_tendency_u_advection_term(u, dx)
        + u_tendency_w_advection_term(u, w, dz)
        + u_tendency_pgf_term(u, pi, theta_base, dx)
    ) * -1

---

In [28]:
@numba.njit()
def theta_p_tendency_u_advection_term(u, theta_p, dx):
    term = np.zeros_like(theta_p)
    for k in range(1, theta_p.shape[0] - 1):
        for i in range(1, theta_p.shape[1] - 1):
            term[k, i] = (
                (u[k, i + 1] + u[k, i]) / 2
                * (theta_p[k, i + 1] - theta_p[k, i - 1]) / (2 * dx)
            )
    return term

In [29]:
@numba.njit()
def theta_p_tendency_w_advection_of_perturbation_term(w, theta_p, dz):
    term = np.zeros_like(theta_p)
    for k in range(1, theta_p.shape[0] - 1):
        for i in range(1, theta_p.shape[1] - 1):
            term[k, i] = (
                (w[k + 1, i] + w[k, i]) / 2
                * (theta_p[k + 1, i] - theta_p[k - 1, i]) / (2 * dz)
            )
    return term

In [30]:
@numba.njit()
def theta_p_tendency_w_advection_of_base_term(w, theta_p, theta_base, dz):
    term = np.zeros_like(theta_p)
    for k in range(1, theta_p.shape[0] - 1):
        for i in range(1, theta_p.shape[1] - 1):
            term[k, i] = (
                (w[k + 1, i] + w[k, i]) / 2
                * (theta_base[k + 1] - theta_base[k - 1]) / (2 * dz)
            )
    return term

In [32]:
@numba.njit()
def theta_p_tendency(u, w, theta_p, theta_base, dx, dz):
    return (
        theta_p_tendency_u_advection_term(u, theta_p, dx)
        + theta_p_tendency_w_advection_of_perturbation_term(w, theta_p, dz)
        + theta_p_tendency_w_advection_of_base_term(w, theta_p, theta_base, dz)
    ) * -1

In [36]:
numba.__version__

'0.58.1'

In [34]:
@numba.njit()
def pi_tendency(u, w, pi, theta_base, rho_base, c_s_sqr, dx, dz):
    term = np.zeros_like(pi)
    for k in range(1, pi.shape[0] - 1):
        for i in range(1, pi.shape[1] - 1):
            term[k, i] = (
                -1 * (c_s_sqr / (c_p * rho_base[k] * theta_base[k]**2))
                * (
                    (rho_base[k] * theta_base[k] * (u[k, i + 1] - u[k, i]) / dx)
                    + (
                        (w[k + 1, i] + w[k, i]) / 2
                        * (rho_base[k + 1] * theta_base[k + 1] - rho_base[k - 1] * theta_base[k - 1]) / (2 * dz)
                    )
                    + (rho_base[k] * theta_base[k] * (w[k + 1, i] - w[k, i]) / dz)
                )
            )
    return term

In [42]:
# Helper functions
# TODO: this should probably go into a separate file!

@numba.njit()
def nondimensional_pressure_hydrostatic(theta, z, pressure_surface):
    pi = np.zeros_like(theta)
    # Start at (w level) surface as given
    pi_sfc = (pressure_surface / p0)**(R_d / c_p)
    # Go down, half level, for u level, using above-surface theta
    pi[0] = pi_sfc + gravity * (z[1] - z[0]) / (2 * c_p * theta[1])
    # Now, integrate upward over full u levels
    for i in range(1, pi.shape[0]):
        theta_at_level = 0.5 * (theta[i] + theta[i - 1])
        pi[i] = pi[i - 1] - gravity * (z[i] - z[i - 1]) / (c_p * theta_at_level)
    return pi

@numba.njit()
def density_from_ideal_gas_law(theta, pi):
    return p0 * pi ** (c_v / R_d) / (R_d * theta)

@numba.njit()
def create_thermal_bubble(amplitude, x, z, x_radius, z_radius, x_center, z_center, theta_base):
    # Coordinates in 2d
    xx = np.broadcast_to(x[None, :], (z.shape[0], x.shape[0]))
    zz = np.broadcast_to(z[:, None], (z.shape[0], x.shape[0]))
    rad = np.sqrt(((zz - z_center) / z_radius)**2 + ((xx - x_center) / x_radius)**2)
    # Create thermal bubble
    theta_p = np.zeros_like(xx)
    for k in range(rad.shape[0]):
        for i in range(rad.shape[1]):
            if rad[k, i] <= 1.0:
                theta_p[k, i] = 0.5 * amplitude * (np.cos(np.pi * rad[k, i]) + 1.0)
    # Create balanced pi, integrating downward from assumed zero at topmost level
    pi = np.zeros_like(th)
    for k in range(rad.shape[0] - 2, -1, -1):
        for i in range(rad.shape[1]):
            integrand_trapz = 0.5 * (
                theta_p[k + 1, i] / theta_base[k + 1]**2
                + theta_p[k, i] / theta_base[k]**2
            )
            pi[k, i] = pi[k + 1, i] - g * (z[k + 1] - z[k]) / c_p * integrand_trapz
    # Return results
    return theta_p, pi

@numba.njit()
def apply_periodic_lateral_zerograd_vertical(a):
    # Bottom and top (no gradient)
    for i in range(0, a.shape[1]):
        a[0, i] = a[1, i]
        a[a.shape[0] - 1, i] = a[a.shape[0] - 2, i]
    # Left and right (mirrored)
    for k in range(1, a.shape[0] - 1):
        a[k, 0] = a[k, a.shape[1] - 2]
        a[k, a.shape[1] - 1] = a[k, 1]
    return a

@numba.njit()
def apply_periodic_lateral_zerow_vertical(a):
    # Bottom and top (fixed zero)
    for i in range(0, a.shape[1]):
        a[0, i] = a[1, i] = 0
        a[a.shape[0] - 1, i] = a[a.shape[0] - 2, i] = 0
    # Left and right (mirrored)
    for k in range(1, a.shape[0] - 1):
        a[k, 0] = a[k, a.shape[1] - 2]
        a[k, a.shape[1] - 1] = a[k, 1]
    return a

In [ ]:
metadata_attrs = {
    'u': {
        'units': 'm/s'
    },
    'w': {
        'units': 'm/s'
    },
    'theta_p': {
        'units': 'K'
    },
    'pi': {
        'units': 'dimensionless'
    },
    'x': {
        'units': 'm'
    },
    'x_stag': {
        'units': 'm'
    },
    'z': {
        'units': 'm'
    },
    'z_stag': {
        'units': 'm'
    },
    't': {
        'units': 's'
    }
}

class ModelDriver:

    coords = {}
    prognostic_arrays = {}
    base_state_arrays = {}
    diagnostic_arrays = {}
    params = {}

    def __init__(self, nx, nz, dx, dz, dt, **kwargs):
        # Set parameters
        self.nx = nx
        self.nz = nz
        self.dx = dx
        self.dz = dz
        self.dt = dt
        self.params.update(kwargs)
        dtype = getattr(self, 'dtype', np.float32)

        # Define arrays
        self.coords['x'] = np.arange(self.nx) * self.dx - self.nx * self.dx / 2
        self.coords['x_stag'] = np.arange(self.nx + 1) * self.dx - (self.nx + 1) * self.dx / 2
        self.coords['z'] = np.arange(self.nz) * self.dz
        self.coords['z_stag'] = np.arange(self.nz + 1) * self.dz - self.dz / 2
        self.prognostic_arrays['u'] = np.zeros((3, nz, nx + 1), dtype=dtype)
        self.prognostic_arrays['w'] = np.zeros((3, nz + 1, nx), dtype=dtype)
        self.prognostic_arrays['theta_p'] = np.zeros((3, nz, nx), dtype=dtype)
        self.prognostic_arrays['pi'] = np.zeros((3, nz, nx), dtype=dtype)
        self.active_prognostic_variables = ['u', 'w', 'theta_p', 'pi']
        self.base_state_arrays['theta_base'] = np.zeros(nz, dtype=dtype)
        self.base_state_arrays['PI_base'] = np.zeros(nz, dtype=dtype)
        self.base_state_arrays['rho_base'] = np.zeros(nz, dtype=dtype)
        ## Todo do we need others??

    def initialize_isentropic_base_state(self, theta, pressure_surface):
        # Set uniform potential temperature
        self.base_state_arrays['theta_base'] = np.full(
            self.base_state_arrays['theta_base'].shape, theta, dtype=self.dtype
        )
        # Calculate pi based on hydrostatic balance (from surface)
        self.base_state_arrays['PI_base'] = nondimensional_pressure_hydrostatic(
            self.base_state_arrays['theta_base'],
            self.coords['z'],
            pressure_surface
        )
        # Calculate density from theta and pi
        self.base_state_arrays['rho_base'] = density_from_ideal_gas_law(
            self.base_state_arrays['theta_base'],
            self.base_state_arrays['PI_base']
        )

    def initialize_warm_bubble(self, amplitude, x_radius, z_radius, z_center):
        if np.min(self.base_state_arrays['theta_base']) <= 0.:
            raise ValueError("Base state theta must be initialized as positive definite")

        # Create thermal bubble (2D)
        theta_p, pi = create_thermal_bubble(
            amplitude, self.coords['x'], self.coords['z'], x_radius, z_radius, 0.0, z_center, 
            self.base_state_arrays['theta_base']
        )
        # Ensure boundary conditions, and add time stacking (future, current, past)
        self.prognostic_arrays['theta_p'] = np.stack([apply_periodic_lateral_zerograd_vertical(theta_p)] * 3)
        self.prognostic_arrays['pi'] = np.stack([apply_periodic_lateral_zerograd_vertical(pi)] * 3)
        
    def prep_new_timestep(self):
        for var in self.active_prognostic_variables:
            # Future-current to current-past
            self.prognostic_arrays[var][0:2] = self.prognostic_arrays[var][1:3]

    def take_first_timestep(self):
        # check for needed parameters and methods
        if not getattr(self, 'c_s_sqr'):
            raise ValueError("Must set squared speed of sound prior to first timestep")
        if not (
            getattr(self, 'u_tendency')
            and getattr(self, 'w_tendency')
            and getattr(self, 'theta_p_tendency')
            and getattr(self, 'pi_tendency')
        ):
            
        # Increment
        self.t_count = 1

        # Integrate forward-in-time
        self.prognostic_arrays['u'][2] = (
            self.prognostic_arrays['u'][1]
            + self.dt * apply_periodic_lateral_zerograd_vertical(self.u_tendency(
                self.prognostic_arrays['u'], self.prognostic_arrays['w'],
                self.prognostic_arrays['pi'], self.base_state_arrays['theta_base'], self.dx, self.dz
            ))
        )
        self.prognostic_arrays['w'][2] = (
            self.prognostic_arrays['w'][1]
            + self.dt * apply_periodic_lateral_zerograd_vertical(self.w_tendency(
                self.prognostic_arrays['u'], self.prognostic_arrays['w'],
                self.prognostic_arrays['pi'], self.prognostic_arrays['theta_p'],
                self.base_state_arrays['theta_base'], self.dx, self.dz
            ))
        )
        self.prognostic_arrays['theta_p'][2] = (
            self.prognostic_arrays['theta_p'][1]
            + self.dt * apply_periodic_lateral_zerograd_vertical(self.theta_p_tendency(
                self.prognostic_arrays['u'], self.prognostic_arrays['w'],
                self.prognostic_arrays['theta_p'], self.base_state_arrays['theta_base'], self.dx, self.dz
            ))
        )
        self.prognostic_arrays['pi'][2] = (
            self.prognostic_arrays['pi'][1]
            + self.dt * apply_periodic_lateral_zerograd_vertical(self.pi_tendency(
                self.prognostic_arrays['u'], self.prognostic_arrays['w'],
                self.prognostic_arrays['pi'], self.base_state_arrays['theta_base'], 
                self.base_state_arrays['rho_base'], self.c_s_sqr, self.dx, self.dz
            ))
        )

        self.prep_new_timestep()

    def take_single_timestep(self):
        # Check if initialized
        if self.t_count == 0:
            raise RuntimeError("Must run initial timestep!")
        self.t_count += 1

        # Integrate leapfrog
        self.prognostic_arrays['u'][2] = (
            self.prognostic_arrays['u'][0]
            + 2 * self.dt * apply_periodic_lateral_zerograd_vertical(self.u_tendency(
                self.prognostic_arrays['u'], self.prognostic_arrays['w'],
                self.prognostic_arrays['pi'], self.base_state_arrays['theta_base'], self.dx, self.dz
            ))
        )
        self.prognostic_arrays['w'][2] = (
            self.prognostic_arrays['w'][0]
            + 2 * self.dt * apply_periodic_lateral_zerograd_vertical(self.w_tendency(
                self.prognostic_arrays['u'], self.prognostic_arrays['w'],
                self.prognostic_arrays['pi'], self.prognostic_arrays['theta_p'],
                self.base_state_arrays['theta_base'], self.dx, self.dz
            ))
        )
        self.prognostic_arrays['theta_p'][2] = (
            self.prognostic_arrays['theta_p'][0]
            + 2 * self.dt * apply_periodic_lateral_zerograd_vertical(self.theta_p_tendency(
                self.prognostic_arrays['u'], self.prognostic_arrays['w'],
                self.prognostic_arrays['theta_p'], self.base_state_arrays['theta_base'], self.dx, self.dz
            ))
        )
        self.prognostic_arrays['pi'][2] = (
            self.prognostic_arrays['pi'][0]
            + 2 * self.dt * apply_periodic_lateral_zerograd_vertical(self.pi_tendency(
                self.prognostic_arrays['u'], self.prognostic_arrays['w'],
                self.prognostic_arrays['pi'], self.base_state_arrays['theta_base'], 
                self.base_state_arrays['rho_base'], self.c_s_sqr, self.dx, self.dz
            ))
        )

        self.prep_new_timestep()

    def current_state(self):
        """Export the prognostic variables, with coordinates, at current time."""
        data_vars = {}
        for var in self.active_prognostic_variables:
            if var == 'u':
                dims = ('t', 'z', 'x_stag')
            elif var == 'w':
                dims = ('t', 'z_stag', 'x')
            else:
                dims = ('t', 'z', 'x')
            data_vars[var] = xr.Variable(dims, self.arrays_prognostic[var][1:2].copy(), metadata_attrs[var])
        data_vars['x'] = xr.Variable('x', self.x, metadata_attrs['x'])
        data_vars['x_stag'] = xr.Variable('x_stag', self.x_stag, metadata_attrs['x_stag'])
        data_vars['z'] = xr.Variable('z', self.z, metadata_attrs['z'])
        data_vars['z_stag'] = xr.Variable('z_stag', self.z_stag, metadata_attrs['z_stag'])
        data_vars['t'] = xr.Variable('t', [self.t_count * self.dt], metadata_attrs['t'])
        return xr.Dataset(data_vars)

---

## Summary

...TODO...

Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!